# Disease Q&A API Cookbook

## Introduction

This notebook breaks down the implementation of a Disease Q&A application. The original application was built as a web app using HTML, CSS, and JavaScript, and we'll convert the core functionality to Python.

## Table of Contents
1. [Setting Up Dependencies](#setting-up-dependencies)
2. [Understanding the Perplexity API](#understanding-the-perplexity-api)
3. [Creating the API Query Function](#creating-the-api-query-function)
4. [Parsing and Displaying Results](#parsing-and-displaying-results)
5. [Building a Simple UI with Jupyter Widgets](#building-a-simple-ui-with-jupyter-widgets)
6. [Full Working Example](#full-working-example)

## Setting Up Dependencies

First, let's install the packages we'll need for this project.

In [ ]:
# Install required packages
!pip install requests ipywidgets

Now let's import the necessary libraries.

In [ ]:
# Import necessary libraries
import requests
import json
import ipywidgets as widgets
from IPython.display import display, HTML

## Understanding the Perplexity API

The original web application uses the Perplexity AI API to generate structured responses about diseases. Let's examine the API endpoint and parameters.

In [ ]:
# API Constants
API_ENDPOINT = "https://api.perplexity.ai/chat/completions"
# Note: You should replace this with your own API key
API_KEY = "YOUR_API_KEY_HERE"  # Replace with your actual API key
MODEL = "sonar-pro"  # The model used in the original application

# Sample headers for the API request
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# Examining the structure of a basic query
sample_payload = {
    "model": MODEL,
    "messages": [
        {
            "role": "user",
            "content": "What is diabetes?"
        }
    ]
}

print(json.dumps(sample_payload, indent=2))

## Creating the API Query Function

Now, let's create a function that sends queries to the Perplexity API, similar to the JavaScript function in the original application.

In [ ]:
def ask_disease_question(question, api_key=API_KEY):
    """
    Query the Perplexity API about a disease and get structured JSON response.
    
    Parameters:
    question (str): The disease-related question
    api_key (str): Your Perplexity API key
    
    Returns:
    dict: JSON response with overview, causes, treatments, and citations
    """
    # Construct a prompt instructing the API to output only valid JSON
    prompt = f"""
You are a medical assistant. Please answer the following question about a disease and provide only valid JSON output.
The JSON object must have exactly four keys: "overview", "causes", "treatments", and "citations".
For example:
{{
  "overview": "A brief description of the disease.",
  "causes": "The causes of the disease.",
  "treatments": "Possible treatments for the disease.",
  "citations": ["https://example.com/citation1", "https://example.com/citation2"]
}}
Now answer this question:
"{question}"
    """.strip()
    
    # Build the payload
    payload = {
        "model": MODEL,
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }
    
    try:
        # Make the API request
        response = requests.post(
            API_ENDPOINT,
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            },
            json=payload
        )
        
        # Check for successful response
        if response.status_code != 200:
            return {
                "error": f"API Error: {response.status_code} - {response.text}"
            }
        
        # Parse the response
        result = response.json()
        
        # Extract the content
        if "choices" in result and len(result["choices"]) > 0:
            content = result["choices"][0]["message"]["content"]
            try:
                # Parse the JSON response
                structured_output = json.loads(content)
                return structured_output
            except json.JSONDecodeError:
                return {
                    "error": f"Failed to parse JSON output from API. Raw output: {content}"
                }
        else:
            return {
                "error": "No answer provided in the response."
            }
    except Exception as e:
        return {
            "error": f"Exception occurred: {str(e)}"
        }

## Parsing and Displaying Results

Let's create functions to display the results in a structured format, similar to the original web application's knowledge card.

In [ ]:
def display_knowledge_card(data):
    """
    Display the disease information in an HTML table format.
    
    Parameters:
    data (dict): The structured response data
    """
    # Check if there was an error
    if "error" in data:
        return HTML(f"""
        <div style="color: red; background-color: #ffeeee; padding: 10px; border-radius: 5px;">
            <strong>Error:</strong> {data["error"]}
        </div>
        """)
    
    # Create an HTML table
    html = """
    <div style="background: white; border: 1px solid #e0e0e0; border-radius: 8px; padding: 1rem; box-shadow: 0 2px 4px rgba(0,0,0,0.1); margin-top: 1.5rem;">
        <h2>Knowledge Card</h2>
        <table style="width: 100%; border-collapse: collapse;">
            <tr>
                <th style="background-color: #fafafa; width: 25%; text-align: left; padding: 0.75rem; border-bottom: 1px solid #e0e0e0;">Overview</th>
                <td style="text-align: left; padding: 0.75rem; border-bottom: 1px solid #e0e0e0;">{}</td>
            </tr>
            <tr>
                <th style="background-color: #fafafa; width: 25%; text-align: left; padding: 0.75rem; border-bottom: 1px solid #e0e0e0;">Causes</th>
                <td style="text-align: left; padding: 0.75rem; border-bottom: 1px solid #e0e0e0;">{}</td>
            </tr>
            <tr>
                <th style="background-color: #fafafa; width: 25%; text-align: left; padding: 0.75rem; border-bottom: 1px solid #e0e0e0;">Treatments</th>
                <td style="text-align: left; padding: 0.75rem; border-bottom: 1px solid #e0e0e0;">{}</td>
            </tr>
        </table>
    </div>
    """.format(
        data.get("overview", "N/A"),
        data.get("causes", "N/A"),
        data.get("treatments", "N/A")
    )
    
    return HTML(html)

def display_citations(data):
    """
    Display the citations in an HTML list format.
    
    Parameters:
    data (dict): The structured response data
    """
    # Check if there was an error
    if "error" in data:
        return None
    
    # Start the HTML for citations
    html = """
    <div style="background: white; border: 1px solid #e0e0e0; border-radius: 8px; padding: 1rem; box-shadow: 0 2px 4px rgba(0,0,0,0.1); margin-top: 1.5rem;">
        <h2>Citations</h2>
        <ul style="list-style-type: disc; padding-left: 20px;">
    """
    
    # Add each citation as a list item
    citations = data.get("citations", [])
    if isinstance(citations, list) and citations:
        for citation in citations:
            html += f'<li style="margin-bottom: 0.5rem;"><a href="{citation}" target="_blank" style="color: #10a37f; text-decoration: none;">{citation}</a></li>'
    else:
        html += '<li style="margin-bottom: 0.5rem;">No citations provided.</li>'
    
    # Close the HTML
    html += """
        </ul>
    </div>
    """
    
    return HTML(html)

## Building a Simple UI with Jupyter Widgets

Now, let's create a simple user interface using Jupyter widgets to mimic the form functionality of the original web application.

In [ ]:
def create_disease_qa_ui():
    """
    Create a simple user interface for the Disease Q&A application.
    """
    # Create the text input widget
    question_input = widgets.Text(
        value='',
        placeholder='Ask a question about a disease (e.g., "What is stroke?")',
        description='Question:',
        layout=widgets.Layout(width='70%')
    )
    
    # Create the submit button
    ask_button = widgets.Button(
        description='Ask',
        button_style='success',
        layout=widgets.Layout(width='100px')
    )
    
    # Create output widgets
    output = widgets.Output()
    loading_output = widgets.Output()
    
    # Define what happens when the button is clicked
    def on_ask_button_clicked(b):
        # Clear previous output
        output.clear_output()
        
        with loading_output:
            loading_output.clear_output()
            print("Loading...")
        
        # Get the question
        question = question_input.value
        
        if not question:
            with output:
                display(HTML('<div style="color: red;">Please enter a question.</div>'))
            loading_output.clear_output()
            return
        
        # Call the API function
        with output:
            try:
                # Call API function (in a real application, you would use the actual API key)
                # For demonstration, we'll use a mock response
                mock_response = {
                    "overview": "This is a mock overview of the disease.",
                    "causes": "These are mock causes of the disease.",
                    "treatments": "These are mock treatments for the disease.",
                    "citations": ["https://example.com/citation1", "https://example.com/citation2"]
                }
                
                # In a real application, you would use:
                # data = ask_disease_question(question, API_KEY)
                data = mock_response
                
                # Display the results
                display(display_knowledge_card(data))
                display(display_citations(data))
            except Exception as e:
                display(HTML(f'<div style="color: red;">Error: {str(e)}</div>'))
            finally:
                # Clear loading message
                loading_output.clear_output()
    
    # Connect the button click to the function
    ask_button.on_click(on_ask_button_clicked)
    
    # Create a horizontal box for the input and button
    input_box = widgets.HBox([question_input, ask_button])
    
    # Display the UI components
    display(widgets.HTML("<h1 style='text-align: center;'>Disease Q&A</h1>"))
    display(input_box)
    display(loading_output)
    display(output)

# Call the function to create and display the UI
create_disease_qa_ui()

## Full Working Example

Here's a complete example that puts everything together. Note: In this example, we're using a mock response instead of making an actual API call to avoid exposing API keys.

In [ ]:
def full_disease_qa_app():
    """
    A complete implementation of the Disease Q&A application.
    """
    # Create the UI components
    question_input = widgets.Text(
        value='',
        placeholder='Ask a question about a disease (e.g., "What is stroke?")',
        description='Question:',
        layout=widgets.Layout(width='70%')
    )
    
    ask_button = widgets.Button(
        description='Ask',
        button_style='success',
        layout=widgets.Layout(width='100px')
    )
    
    output = widgets.Output()
    loading_output = widgets.Output()
    
    # Function to handle the button click
    def on_ask_button_clicked(b):
        # Clear previous output
        output.clear_output()
        
        with loading_output:
            loading_output.clear_output()
            print("Loading...")
        
        # Get the question
        question = question_input.value
        
        if not question:
            with output:
                display(HTML('<div style="color: red;">Please enter a question.</div>'))
            loading_output.clear_output()
            return
        
        # Process the question
        with output:
            try:
                # In a real application, you would use:
                # data = ask_disease_question(question, "YOUR_ACTUAL_API_KEY")
                
                # For demonstration, we'll use mock data for common diseases
                mock_data = {}
                lower_question = question.lower()
                
                if "diabetes" in lower_question:
                    mock_data = {
                        "overview": "Diabetes is a chronic condition that affects how your body turns food into energy. There are several types, with Type 1 and Type 2 being the most common.",
                        "causes": "Type 1 diabetes is caused by an autoimmune reaction where the body attacks insulin-producing cells. Type 2 diabetes develops when the body becomes resistant to insulin or doesn't produce enough insulin.",
                        "treatments": "Treatments include monitoring blood sugar levels, insulin therapy, medication, healthy eating, regular exercise, and maintaining a healthy weight.",
                        "citations": ["https://www.cdc.gov/diabetes/", "https://www.mayoclinic.org/diseases-conditions/diabetes/"]
                    }
                elif "stroke" in lower_question:
                    mock_data = {
                        "overview": "A stroke occurs when blood supply to part of the brain is interrupted or reduced, preventing brain tissue from getting oxygen and nutrients.",
                        "causes": "Causes include blocked arteries (ischemic stroke) or leaking/bursting blood vessels (hemorrhagic stroke).",
                        "treatments": "Treatments depend on the type of stroke and may include medication to break up clots, surgery, and rehabilitation.",
                        "citations": ["https://www.stroke.org/", "https://www.mayoclinic.org/diseases-conditions/stroke/"]
                    }
                elif "heart attack" in lower_question or "cardiac arrest" in lower_question:
                    mock_data = {
                        "overview": "A heart attack occurs when blood flow to part of the heart is blocked, causing damage to the heart muscle.",
                        "causes": "Most heart attacks are caused by coronary artery disease, which involves the buildup of plaque in the arteries.",
                        "treatments": "Treatments include medications, surgery, and lifestyle changes to improve heart health.",
                        "citations": ["https://www.heart.org/", "https://www.mayoclinic.org/diseases-conditions/heart-attack/"]
                    }
                else:
                    mock_data = {
                        "overview": f"Information about {question} would be retrieved from the API in a real application.",
                        "causes": "This is a mock response. In a real application, specific causes would be listed here.",
                        "treatments": "This is a mock response. In a real application, specific treatments would be listed here.",
                        "citations": ["https://example.com/medical-resource1", "https://example.com/medical-resource2"]
                    }
                
                # Display the results
                display(display_knowledge_card(mock_data))
                display(display_citations(mock_data))
            except Exception as e:
                display(HTML(f'<div style="color: red;">Error: {str(e)}</div>'))
            finally:
                # Clear loading message
                loading_output.clear_output()
    
    # Connect the button click to the function
    ask_button.on_click(on_ask_button_clicked)
    
    # Create a horizontal box for the input and button
    input_box = widgets.HBox([question_input, ask_button])
    
    # Set page styling
    display(HTML("""
    <style>
    .container {
        max-width: 800px;
        margin: 2rem auto;
        font-family: Helvetica, Arial, sans-serif;
    }
    h1 {
        text-align: center;
        color: #111;
        margin-bottom: 1rem;
    }
    </style>
    
    <div class="container">
        <h1>Disease Q&A</h1>
    </div>
    """))
    
    # Display the UI components
    display(input_box)
    display(loading_output)
    display(output)

# Call the function to create and display the app
full_disease_qa_app()

## Conclusion

This notebook has demonstrated how to convert a web-based Disease Q&A application into a Python notebook. We've covered:

1. Setting up the necessary dependencies
2. Understanding the Perplexity API structure
3. Creating functions to query the API
4. Parsing and displaying the structured results
5. Building a simple UI with Jupyter widgets

In a production environment, you would need to:
- Replace the mock API responses with actual API calls
- Use your own API key
- Add error handling for various edge cases
- Consider rate limiting and other API restrictions

This notebook serves as a starting point for building more complex applications that leverage AI APIs to provide structured information.